In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [155]:
class PCR:
    def __init__(self, population_as_list, n_primers):
        self.max_array_len = max([len(x) for x in population_as_list])
        
        self.f_population = np.ones((len(population_as_list), self.max_array_len + 2)) * -1
        
        for i, array in enumerate(population_as_list):
            self.f_population[i][1:1 + len(array)] = array  
        self.b_population = np.copy(self.f_population)
        
        self.n_f_primes = n_primers
        self.n_b_primers = n_primers
        
        self.n_spacers = len(set([x for y in population_as_list for x in y]))
        
        self.p = [1 / self.max_array_len, 0.1 / self.max_array_len, 1 - 1/ self.max_array_len - 0.1/self.max_array_len ]
        
    def _increase_populations(self, n_f, n_b):
        self.f_population = np.vstack((self.f_population, np.zeros((n_f, self.max_array_len + 2))))
        self.b_population = np.vstack((self.b_population, np.zeros((n_b, self.max_array_len + 2))))
        
    def _make_annealing_masks(self):
        #print(self.b_population.size)
        f_annealing_mask = np.random.choice([0,1,2], self.f_population.shape, p = self.p)#.reshape(self.f_population.shape)
        b_annealing_mask = np.random.choice([0,1,2], self.b_population.shape, p = self.p)#.reshape(self.b_population.shape)
        
        f_annealing_mask[self.f_population == -1] = 2
        b_annealing_mask[self.b_population == -1] = 2
        return f_annealing_mask, b_annealing_mask, (f_annealing_mask == 0).sum(), (b_annealing_mask == 0).sum()
    
        
    def _make_annealing_linkage(self, f_annealing_mask, b_annealing_mask):
        f_indeces_0 = np.vstack(np.where(f_annealing_mask == 0)).T
        nonzero_b = (self.b_population.sum(1) != 0).sum()
        b_paired_0 = np.vstack((np.arange(nonzero_b,
                                          nonzero_b + f_indeces_0.shape[0]),
                                np.ones(f_indeces_0.shape[0]) * (self.max_array_len + 1))).T
        
        print(self.f_population.shape)
        b_indeces_0 = np.vstack(np.where(b_annealing_mask == 0)).T
        nonzero_f = (self.f_population.sum(1) != 0).sum()
        f_paired_0 = np.vstack((np.arange(nonzero_f,
                                          nonzero_f + b_indeces_0.shape[0]),
                                np.zeros(b_indeces_0.shape[0]))).T
        
        
        f_indeces_1 = np.vstack(np.where(f_annealing_mask == 1)).T
        b_indeces_1 = np.vstack(np.where(b_annealing_mask == 1)).T
        np.random.shuffle(f_indeces_1)
        np.random.shuffle(b_indeces_1)
        f_indeces_1 = f_indeces_1[:min(f_indeces_1.shape[0], b_indeces_1.shape[0])]
        b_indeces_1 = b_indeces_1[:min(f_indeces_1.shape[0], b_indeces_1.shape[0])]
        

        return np.vstack((f_indeces_0, f_paired_0, f_indeces_1)).astype(int), \
                np.vstack((b_paired_0, b_indeces_0, b_indeces_1)).astype(int)

    def _copy(self, index_f, index_b):
        if index_f[1] self.max_array_len + 2 and index_f[1]
        
        f_copy_part = self.b_population[index_b[0], index_b[1] :]
        b_copy_part = self.f_population[index_f[0], :index_f[1]]
        
        
        
        return f_copy_part, b_copy_part

In [156]:
test_p = [[1,2,3], [2, 4, 5, 6]]

In [157]:
pcr = PCR(test_p, 19)


In [158]:
pcr.f_population

array([[-1.,  1.,  2.,  3., -1., -1.],
       [-1.,  2.,  4.,  5.,  6., -1.]])

In [159]:
fm, bm, nf,nb = pcr._make_annealing_masks()
pcr._increase_populations(nf, nb)

In [160]:
fm

array([[2, 2, 2, 0, 2, 2],
       [2, 1, 0, 2, 2, 2]])

In [161]:
bm

array([[2, 2, 0, 2, 2, 2],
       [2, 2, 2, 0, 0, 2]])

In [162]:
fi, bi = pcr._make_annealing_linkage(fm, bm)

(4, 6)


In [163]:
fi

array([[0, 3],
       [1, 2],
       [2, 0],
       [3, 0],
       [4, 0]])

In [164]:
bi

array([[2, 5],
       [3, 5],
       [0, 2],
       [1, 3],
       [1, 4]])

In [124]:
pcr.b_population

array([[-1.,  1.,  2.,  3., -1., -1.],
       [-1.,  2.,  4.,  5.,  6., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [125]:
pcr.f_population

array([[-1.,  1.,  2.,  3., -1., -1.],
       [-1.,  2.,  4.,  5.,  6., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [165]:
pcr._copy(fi[2], bi[2])

(array([ 2.,  3., -1., -1.]), array([], dtype=float64))

In [67]:
pcr.b_population[bi[0][0], bi[0][1] :]

IndexError: index 2 is out of bounds for axis 0 with size 2

In [57]:
bi[0][1]

-1